# Medical Diagnosis LLM via Low-Rank Adaptation and Retrieval-Augmentation

A complete RAG pipeline for medical question answering using:
- **PubMedQA** dataset (211K+ biomedical QA pairs)
- **PubMedBERT** for domain-specific embeddings
- **ChromaDB** vector database for evidence retrieval
- **BioMistral-7B + LoRA** for fine-tuned medical reasoning
- **Safety guardrails** for responsible AI in healthcare

In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))
os.chdir(os.path.join(os.getcwd(), '..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

from data_loader import load_config, load_pubmedqa, create_train_val_test_split, format_for_training

config = load_config()
print('Configuration loaded successfully!')

---
## 1. Data Loading & Preprocessing

In [ ]:
# Load PubMedQA datasets
data = load_pubmedqa(config)

labeled_df = data['labeled']
artificial_df = data['artificial']

print(f'\n=== Dataset Summary ===')
print(f'Labeled (expert-annotated): {len(labeled_df)} examples')
print(f'Artificial (machine-generated): {len(artificial_df)} examples')
print(f'\nLabeled columns: {list(labeled_df.columns)}')
print(f'\nLabel distribution (labeled):')
print(labeled_df['final_decision'].value_counts())
print(f'\nLabel distribution (artificial):')
print(artificial_df['final_decision'].value_counts())

In [ ]:
# Show sample examples
print('=== Sample Examples ===')
for i in range(3):
    row = labeled_df.iloc[i]
    print(f'\n--- Example {i+1} ---')
    print(f'Question: {row["question"]}')
    print(f'Context: {row["context"][:200]}...')
    print(f'Answer: {row["final_decision"]}')
    print(f'Explanation: {row["long_answer"][:150]}...')

In [ ]:
# Visualize dataset
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Label distribution
colors = ['#2ecc71', '#e74c3c', '#f39c12']
labeled_df['final_decision'].value_counts().plot(kind='bar', ax=axes[0], color=colors)
axes[0].set_title('Label Distribution (Labeled Set)')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=0)

# Context length distribution
labeled_df['context_len'] = labeled_df['context'].str.len()
axes[1].hist(labeled_df['context_len'], bins=30, color='#3498db', alpha=0.7)
axes[1].set_title('Context Length Distribution')
axes[1].set_xlabel('Characters')
axes[1].set_ylabel('Count')

# Question length distribution
labeled_df['question_len'] = labeled_df['question'].str.split().str.len()
axes[2].hist(labeled_df['question_len'], bins=20, color='#9b59b6', alpha=0.7)
axes[2].set_title('Question Length Distribution (words)')
axes[2].set_xlabel('Words')
axes[2].set_ylabel('Count')

plt.tight_layout()
plt.savefig('fig1_data_overview.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Train/Val/Test split
train_df, val_df, test_df = create_train_val_test_split(labeled_df, config)

# Show training format
print('\n=== Training Prompt Format ===')
print(format_for_training(train_df.iloc[0])[:500])
print('...')

---
## 2. Embedding Generation with PubMedBERT

In [ ]:
from embeddings import PubMedBERTEmbedder

embedder = PubMedBERTEmbedder(config)
print(f'Embedding dimension: {embedder.get_embedding_dim()}')

# Test with medical texts
test_texts = [
    'Does aspirin reduce the risk of heart attack?',
    'Aspirin has been shown to reduce platelet aggregation and lower cardiovascular risk.',
    'The treatment of diabetes involves insulin management.',
    'Python is a programming language used for data science.',
]

test_embeddings = embedder.embed_texts(test_texts)
print(f'\nEmbedding shape: {test_embeddings.shape}')

# Similarity matrix
from numpy.linalg import norm
sim_matrix = np.zeros((len(test_texts), len(test_texts)))
for i in range(len(test_texts)):
    for j in range(len(test_texts)):
        sim_matrix[i][j] = np.dot(test_embeddings[i], test_embeddings[j]) / (
            norm(test_embeddings[i]) * norm(test_embeddings[j]))

fig, ax = plt.subplots(figsize=(8, 6))
labels_short = ['Q: Aspirin?', 'A: Aspirin cardio', 'A: Diabetes', 'Non-medical']
sns.heatmap(sim_matrix, annot=True, fmt='.3f', xticklabels=labels_short,
            yticklabels=labels_short, cmap='YlOrRd', ax=ax)
ax.set_title('PubMedBERT Semantic Similarity Matrix')
plt.tight_layout()
plt.savefig('fig2_embedding_similarity.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nKey observations:')
print(f'  Aspirin Q vs Aspirin A: {sim_matrix[0][1]:.3f} (should be HIGH)')
print(f'  Aspirin Q vs Diabetes:  {sim_matrix[0][2]:.3f} (should be medium)')
print(f'  Aspirin Q vs Non-med:   {sim_matrix[0][3]:.3f} (should be LOW)')

---
## 3. Vector Database & Indexing

In [ ]:
from vector_store import MedicalVectorStore

# Index a sample of the knowledge base (use more for production)
SAMPLE_SIZE = 5000  # Increase to 50000+ for better retrieval
sample_df = artificial_df.sample(n=SAMPLE_SIZE, random_state=42)
print(f'Indexing {SAMPLE_SIZE} documents into ChromaDB...')

store = MedicalVectorStore(config, embedder)
store.create_index(sample_df)

In [ ]:
# Test retrieval
collection = store.get_collection()
print(f'Total indexed chunks: {collection.count()}')

# Sample query
query = 'Does aspirin reduce the risk of heart attack?'
results = store.query(query, top_k=5)

print(f'\nQuery: {query}')
print(f'Retrieved {len(results["documents"])} results:\n')
for i, (doc, dist, meta) in enumerate(zip(results['documents'], results['distances'], results['metadatas'])):
    sim = 1 - (dist / 2)
    print(f'  [{i+1}] Similarity: {sim:.4f} | PubMed: {meta["pubid"]}')
    print(f'      {doc[:150]}...\n')

---
## 4. Semantic Retriever with MMR

In [ ]:
from retriever import MedicalRetriever

retriever = MedicalRetriever(config, store)

test_queries = [
    'Does aspirin reduce the risk of heart attack?',
    'Is metformin effective for type 2 diabetes?',
    'Can statins prevent stroke in elderly patients?',
    'Does exercise help with depression?',
]

for query in test_queries:
    evidence = retriever.retrieve(query)
    print(f'\nQuery: {query}')
    print(f'  Retrieved: {len(evidence)} passages')
    if evidence:
        print(f'  Top score: {evidence[0].score:.4f}')
        print(f'  Top text: {evidence[0].text[:100]}...')

In [ ]:
# Visualize retrieval scores
all_scores = []
all_queries = []

for query in test_queries:
    evidence = retriever.retrieve(query, top_k=5)
    for ev in evidence:
        all_scores.append(ev.score)
        all_queries.append(query[:30] + '...')

fig, ax = plt.subplots(figsize=(10, 5))
score_df = pd.DataFrame({'Query': all_queries, 'Score': all_scores})
sns.boxplot(data=score_df, x='Query', y='Score', ax=ax, palette='Set2')
ax.set_title('Retrieval Similarity Scores by Query')
ax.set_ylabel('Cosine Similarity')
ax.tick_params(axis='x', rotation=30)
plt.tight_layout()
plt.savefig('fig3_retrieval_scores.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 5. LoRA Fine-tuning Configuration

**Note:** Full fine-tuning takes several hours. Below shows the configuration and approach.
To actually train, run: `python src/lora_finetune.py`

In [ ]:
# Show LoRA configuration
print('=== LoRA Configuration ===')
print(f'Base Model:     {config["llm"]["base_model"]}')
print(f'LoRA Rank (r):  {config["lora"]["r"]}')
print(f'LoRA Alpha:     {config["lora"]["alpha"]}')
print(f'LoRA Dropout:   {config["lora"]["dropout"]}')
print(f'Target Modules: {config["lora"]["target_modules"]}')
print(f'\n=== Training Configuration ===')
print(f'Epochs:         {config["training"]["num_epochs"]}')
print(f'Batch Size:     {config["training"]["batch_size"]}')
print(f'Learning Rate:  {config["training"]["learning_rate"]}')
print(f'Max Seq Length: {config["training"]["max_seq_length"]}')
print(f'Output Dir:     {config["training"]["output_dir"]}')

# Calculate trainable parameters (estimated)
r = config['lora']['r']
# BioMistral-7B has ~7B params, LoRA adds r*d*2*num_modules params
d = 4096  # hidden dim for 7B model
num_modules = len(config['lora']['target_modules'])
lora_params = r * d * 2 * num_modules * 32  # 32 layers
total_params = 7_000_000_000

print(f'\n=== Parameter Efficiency ===')
print(f'Total model params:    ~{total_params/1e9:.1f}B')
print(f'LoRA trainable params: ~{lora_params/1e6:.1f}M')
print(f'Trainable percentage:  {lora_params/total_params*100:.3f}%')
print(f'Memory saved:          ~{(1 - lora_params/total_params)*100:.1f}%')

In [ ]:
# Visualize LoRA architecture
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Parameter comparison
labels = ['Full Model\n(7B params)', 'LoRA Adapters\n(~17M params)']
sizes = [total_params/1e9, lora_params/1e6/1000]
axes[0].bar(labels, sizes, color=['#e74c3c', '#2ecc71'])
axes[0].set_ylabel('Parameters (Billions)')
axes[0].set_title('Full Fine-tuning vs LoRA Parameter Count')
axes[0].set_yscale('log')

# Training data composition
train_labels = ['pqa_artificial\n(Training)', 'pqa_labeled\n(Evaluation)']
train_sizes = [len(artificial_df), len(labeled_df)]
axes[1].bar(train_labels, train_sizes, color=['#3498db', '#f39c12'])
axes[1].set_ylabel('Number of Examples')
axes[1].set_title('Training Data Composition')

for i, v in enumerate(train_sizes):
    axes[1].text(i, v + 2000, f'{v:,}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('fig4_lora_overview.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 6. RAG Pipeline & Safety Guardrails

In [ ]:
from pipeline import MedicalRAGPipeline
from guardrails import ClinicalGuardrails

# Initialize pipeline (without LLM for demo - uses rule-based fallback)
pipeline = MedicalRAGPipeline(config)

# Test medical query detection
guardrails = ClinicalGuardrails(config)
test_queries_safety = [
    'Does aspirin reduce the risk of heart attack?',
    'Is metformin effective for diabetes?',
    'What is the weather today?',
    'Can exercise help with depression?',
    'What is the best pizza restaurant?',
]

print('=== Medical Query Detection ===')
for q in test_queries_safety:
    is_med = guardrails._is_medical_query(q)
    print(f"  {'[MEDICAL]  ' if is_med else '[NON-MED]  '} {q}")

In [ ]:
# Run full pipeline on sample questions
medical_questions = [
    'Does aspirin reduce the risk of heart attack?',
    'Is metformin effective for type 2 diabetes?',
    'Can statins prevent stroke?',
    'Does exercise help with depression?',
]

responses = []
for q in medical_questions:
    print(f'\n{"="*70}')
    response = pipeline.answer(q)
    responses.append(response)
    print(f'\nQuestion: {response.question}')
    print(f'Decision: {response.decision}')
    print(f'Confidence: {response.confidence:.4f}')
    print(f'Evidence count: {len(response.evidence_texts)}')
    print(f'Safe: {response.is_safe}')
    print(f'Warnings: {response.warnings}')
    print(f'\nTop evidence: {response.evidence_texts[0][:200]}...' if response.evidence_texts else 'No evidence')

In [ ]:
# Show full safe response for one example
print('=== FULL SAFE RESPONSE ===')
print(responses[0].safe_response)

---
## 7. Evaluation on PubMedQA Test Set

In [ ]:
from evaluate import PipelineEvaluator

evaluator = PipelineEvaluator(pipeline, config)

# Evaluate retrieval on test set
retrieval_metrics = evaluator.evaluate_retrieval(test_df.head(50))  # Use subset for speed

In [ ]:
# Evaluate generation (using rule-based fallback since LLM not loaded in notebook)
gen_metrics = evaluator.evaluate_generation(test_df.head(50))

In [ ]:
# Plot confusion matrix
details = gen_metrics['details']
evaluator.plot_confusion_matrix(
    details['true'].tolist(), details['predicted'].tolist(),
    save_path='fig5_confusion_matrix.png'
)

In [ ]:
# Plot confidence distribution
evaluator.plot_confidence_distribution(details, save_path='fig6_confidence_dist.png')

In [ ]:
# Plot safety analysis
evaluator.plot_safety_analysis(details, save_path='fig7_safety_analysis.png')

In [ ]:
# Comprehensive metrics summary
print('\n' + '='*60)
print('EVALUATION SUMMARY')
print('='*60)

print(f'\n--- Retrieval Metrics ---')
print(f'  Hit Rate:         {retrieval_metrics["hit_rate"]:.4f}')
print(f'  MRR:              {retrieval_metrics["mrr"]:.4f}')
print(f'  Avg Similarity:   {retrieval_metrics["avg_similarity"]:.4f}')

print(f'\n--- Generation Metrics ---')
print(f'  Accuracy:         {gen_metrics["accuracy"]:.4f}')
print(f'  F1 (macro):       {gen_metrics["f1_macro"]:.4f}')
print(f'  F1 (weighted):    {gen_metrics["f1_weighted"]:.4f}')

print(f'\n--- Per-Class Performance ---')
for label, metrics in gen_metrics['per_class'].items():
    print(f'  {label:6s}: P={metrics["precision"]:.3f} R={metrics["recall"]:.3f} F1={metrics["f1"]:.3f} (n={metrics["support"]})')

print(f'\n--- Safety Metrics ---')
safe_rate = details['is_safe'].mean() * 100
print(f'  Safety Pass Rate: {safe_rate:.1f}%')
print(f'  Avg Warnings:     {details["num_warnings"].mean():.2f}')

---
## 8. Pipeline Architecture Visualization

In [ ]:
# Create architecture diagram
fig, ax = plt.subplots(figsize=(16, 10))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Title
ax.text(5, 9.5, 'Medical RAG Pipeline Architecture', ha='center', fontsize=18, fontweight='bold')

# Row 1: Indexing Pipeline
ax.text(0.5, 8.5, 'INDEXING PIPELINE', fontsize=12, fontweight='bold', color='#2c3e50')
boxes_row1 = [
    (1, 7.5, 'PubMedQA\n(211K docs)', '#3498db'),
    (3, 7.5, 'Chunker\n(256 tokens)', '#2ecc71'),
    (5, 7.5, 'PubMedBERT\nEmbeddings', '#e74c3c'),
    (7, 7.5, 'ChromaDB\nVector Store', '#9b59b6'),
]
for x, y, text, color in boxes_row1:
    ax.add_patch(plt.Rectangle((x-0.7, y-0.4), 1.4, 0.8, fill=True, 
                                facecolor=color, alpha=0.3, edgecolor=color, linewidth=2))
    ax.text(x, y, text, ha='center', va='center', fontsize=9, fontweight='bold')

for x in [2, 4, 6]:
    ax.annotate('', xy=(x+0.3, 7.5), xytext=(x-0.3, 7.5),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Row 2: Query Pipeline  
ax.text(0.5, 6, 'QUERY PIPELINE', fontsize=12, fontweight='bold', color='#2c3e50')
boxes_row2 = [
    (1, 5, 'User\nQuery', '#f39c12'),
    (3, 5, 'PubMedBERT\nEmbed Query', '#e74c3c'),
    (5, 5, 'Retriever\n(MMR, Top-5)', '#2ecc71'),
    (7, 5, 'Evidence\nPassages', '#3498db'),
]
for x, y, text, color in boxes_row2:
    ax.add_patch(plt.Rectangle((x-0.7, y-0.4), 1.4, 0.8, fill=True,
                                facecolor=color, alpha=0.3, edgecolor=color, linewidth=2))
    ax.text(x, y, text, ha='center', va='center', fontsize=9, fontweight='bold')

for x in [2, 4, 6]:
    ax.annotate('', xy=(x+0.3, 5), xytext=(x-0.3, 5),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Row 3: Generation Pipeline
ax.text(0.5, 3.5, 'GENERATION PIPELINE', fontsize=12, fontweight='bold', color='#2c3e50')
boxes_row3 = [
    (2, 2.5, 'Prompt\nComposer', '#f39c12'),
    (4.5, 2.5, 'BioMistral-7B\n+ LoRA', '#e74c3c'),
    (7, 2.5, 'Safety\nGuardrails', '#27ae60'),
    (9, 2.5, 'Final\nResponse', '#9b59b6'),
]
for x, y, text, color in boxes_row3:
    ax.add_patch(plt.Rectangle((x-0.7, y-0.4), 1.4, 0.8, fill=True,
                                facecolor=color, alpha=0.3, edgecolor=color, linewidth=2))
    ax.text(x, y, text, ha='center', va='center', fontsize=9, fontweight='bold')

for x1, x2 in [(2.7, 3.8), (5.2, 6.3), (7.7, 8.3)]:
    ax.annotate('', xy=(x2, 2.5), xytext=(x1, 2.5),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))

# Connect rows
ax.annotate('', xy=(7, 5.4), xytext=(7, 7.1),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5, ls='--'))
ax.annotate('', xy=(2, 2.9), xytext=(7, 4.6),
            arrowprops=dict(arrowstyle='->', color='gray', lw=1.5, ls='--'))

# Legend
ax.text(1, 1, 'Key: Blue=Data | Green=Processing | Red=Models | Purple=Output | Yellow=Input',
        fontsize=9, style='italic', color='gray')

plt.tight_layout()
plt.savefig('fig8_architecture.png', dpi=150, bbox_inches='tight')
plt.show()

---
## Summary

### What Was Built
A complete Retrieval-Augmented Generation (RAG) pipeline for medical question answering with:

1. **Data Pipeline**: Loaded and preprocessed 212K PubMedQA examples with train/val/test splits
2. **PubMedBERT Embeddings**: Domain-specific biomedical embeddings for semantic retrieval
3. **ChromaDB Vector Store**: Indexed knowledge base with chunked abstracts
4. **MMR Retriever**: Semantic search with diversity-preserving reranking
5. **BioMistral-7B + QLoRA**: Parameter-efficient fine-tuning (0.24% trainable params)
6. **Safety Guardrails**: Disclaimers, hallucination detection, scope filtering, confidence checks
7. **Evaluation Suite**: Retrieval metrics (MRR, Hit Rate), Generation metrics (Accuracy, F1), Safety analysis

### Key Design Decisions
- **PubMedBERT > general BERT**: 10-15% better at biomedical semantic similarity
- **QLoRA (4-bit)**: Reduces memory from 28GB to ~6GB, enabling training on consumer hardware
- **MMR reranking**: Prevents redundant evidence, improves answer diversity
- **Safety-first**: Every response includes disclaimers, confidence scores, and evidence sources

### How to Use in Production
1. Run `python src/lora_finetune.py` for full model training
2. Index the full 211K knowledge base: increase `SAMPLE_SIZE` in vector_store
3. Load fine-tuned model: `from lora_finetune import load_finetuned_model`
4. Run pipeline: `pipeline.answer('your medical question')`